# Implementace expertního systému pro zvířata

Příklad z [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

V tomto příkladu implementujeme jednoduchý znalostní systém k určení zvířete na základě některých fyzických znaků. Systém lze znázornit následujícím stromem AND-OR (jedná se o část celého stromu, můžeme snadno přidat další pravidla):

![](../../../../../../translated_images/cs/AND-OR-Tree.5592d2c70187f283.webp)


## Naše vlastní shell expertních systémů s reverzním inferenčním mechanismem

Zkusme definovat jednoduchý jazyk pro reprezentaci znalostí založený na produkčních pravidlech. Budeme používat třídy Pythonu jako klíčová slova pro definici pravidel. Existují v podstatě 3 typy tříd:
* `Ask` představuje otázku, kterou je třeba položit uživateli. Obsahuje sadu možných odpovědí.
* `If` představuje pravidlo a je to jen syntaktický cukr pro uložení obsahu pravidla.
* `AND`/`OR` jsou třídy, které reprezentují AND/OR větve stromu. Ukládají jen seznam argumentů uvnitř. Pro zjednodušení kódu je veškerá funkčnost definována v rodičovské třídě `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

V našem systému by pracovní paměť obsahovala seznam **faktů** jako **páry atribut-hodnota**. Znalostní báze může být definována jako jedno velké slovníkové pole, které mapuje akce (nové fakty, které by měly být vloženy do pracovní paměti) na podmínky, vyjádřené jako výrazy AND-OR. Některé fakty lze také `Ask`-nout.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Pro provedení zpětného odvozování definujeme třídu `Knowledgebase`. Bude obsahovat:
* Pracovní `memory` - slovník, který mapuje atributy na hodnoty
* Pravidla Knowledgebase `rules` ve formátu definovaném výše

Dvě hlavní metody jsou:
* `get` pro získání hodnoty atributu, provádějící odvozování, je-li to nutné. Například `get('color')` získá hodnotu atributu barva (v případě potřeby se zeptá a uloží hodnotu pro pozdější použití do pracovní paměti). Pokud se zeptáme `get('color:blue')`, zeptá se na barvu a poté vrátí hodnotu `y`/`n` podle barvy.
* `eval` provádí skutečné odvozování, tj. prochází AND/OR strom, hodnotí podcíle atd.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Nyní si definujme naši databázi znalostí o zvířatech a proveďme konzultaci. Všimněte si, že tento dotaz vás bude klást otázky. Můžete odpovídat zadáním `y`/`n` pro otázky ano/ne nebo zadáním čísla (0..N) u otázek s delšími vícečetnými možnostmi odpovědí.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Použití Experta pro dopředné odvozování

V následujícím příkladu se pokusíme implementovat dopředné odvozování pomocí jedné z knihoven pro reprezentaci znalostí, [Experta](https://github.com/nilp0inter/experta). **Experta** je knihovna pro vytváření systémů dopředného odvozování v Pythonu, která je navržena tak, aby byla podobná klasickému starému systému [CLIPS](http://www.clipsrules.net/index.html).

Dopředné odvozování bychom mohli také implementovat sami bez větších problémů, ale naivní implementace obvykle nejsou příliš efektivní. Pro efektivnější hledání pravidel se používá speciální algoritmus [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Náš systém definujeme jako třídu, která dědí od `KnowledgeEngine`. Každé pravidlo je definováno samostatnou funkcí s anotací `@Rule`, která určuje, kdy má být pravidlo spuštěno. Uvnitř pravidla můžeme přidat nové fakta pomocí funkce `declare`, a přidání těchto faktů způsobí, že forward inference engine zavolá další pravidla.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Jakmile jsme definovali znalostní bázi, naplníme naši pracovní paměť některými počátečními fakty a poté zavoláme metodu `run()`, abychom provedli inferenci. Výsledkem je, že do pracovní paměti jsou přidány nové odvozené fakty, včetně konečného faktu o zvířeti (pokud jsme nastavili všechny počáteční fakta správně).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Prohlášení o omezené odpovědnosti**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože usilujeme o přesnost, vězte prosím, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho mateřském jazyce by měl být považován za závazný zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Nejsme odpovědní za jakékoli nedorozumění nebo nesprávné výklady vyplývající z použití tohoto překladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
